In [6]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import cred

scope = "user-read-recently-played"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cred.client_ID, client_secret=cred.client_SECRET, redirect_uri=cred.redirect_url, scope=scope))

# results = sp.current_user_recently_played()
# for idx, item in enumerate(results['items']):
#     track = item['track']
#     print(idx, track['artists'][0]['name'], " – ", track['name'])

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [10]:
df = analyze_playlist('Matthew Lim','30cdY3LXb1nZBdTY05wEXj')
df.head()
# csv
# df.to_csv("dataframe.csv", index = False)

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Mac Ayres,Drive Slow,Easy,58dSdjfEYNSxte1aNVxuNf,0.722,0.282,5,-7.484,1,0.106,0.00557,0.147,0.217,91.367,314358,4
1,Daniel Caesar,Freudian,Best Part (feat. H.E.R.),1RMJOxR6GRPsBHL8qeC2ux,0.524,0.364,7,-10.209,1,0.0367,0,0.1,0.436,75.24,209831,4
2,Bruno Major,A Song For Every Moon,Easily,2k9N4caeCIJLOWwWwssrEM,0.772,0.256,7,-8.545,0,0.0481,0.00612,0.144,0.357,118.902,210240,3
3,keshi,skeletons,skeletons,2BuJrxYKhYky20dQqTlobO,0.719,0.265,2,-12.782,1,0.0754,0.0865,0.108,0.273,80.021,152480,4
4,Kendrick Lamar,DAMN.,LOVE. FEAT. ZACARI.,6PGoSes0D9eUDeeAafB2As,0.8,0.585,10,-7.343,1,0.0924,0,0.153,0.779,126.058,213400,4


## Playlists Classified as 'Happy'

In [13]:
mood_booster = analyze_playlist('spotify','37i9dQZF1DX3rxVfibe1L0')
mood_booster.to_csv("mood_booster.csv", index = False)

In [15]:
great_day = analyze_playlist('spotify','37i9dQZF1DX7KNKjOK0o75')
great_day.to_csv("great_day.csv", index = False)

In [16]:
good_vibes = analyze_playlist('spotify','37i9dQZF1DWYBO1MoTDhZI')
good_vibes.to_csv("good_vibes.csv", index = False)

In [24]:
happy = pd.concat([mood_booster,great_day,good_vibes],ignore_index=True)
happy.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Post Malone,Twelve Carat Toothache,I Like You (A Happier Song) (with Doja Cat),0O6u0VJ46W86TxN9wgyqDj,0.733,0.67,5,-6.009,1,0.0751,0,0.121,0.472,100.964,192841,4
1,NEIKED,Better Days (NEIKED x Mae Muller x Polo G),Better Days (NEIKED x Mae Muller x Polo G),6f5ExP43esnvdKPddwKXJH,0.717,0.671,0,-5.077,0,0.0337,0.000003,0.0921,0.699,110.054,160657,4
2,Sabrina Carpenter,emails i can't send,Nonsense,6dgUya35uo964z7GZXM07g,0.74,0.697,8,-4.912,1,0.034,0,0.224,0.732,138.992,163648,4
3,Khalid,Skyline,Skyline,1QxIFMs4M8qd5cbJJu80v4,0.733,0.653,11,-6.094,1,0.0391,0.0218,0.184,0.752,119.996,182895,5
4,Taylor Swift,Midnights,Lavender Haze,5jQI2r1RdgtuT8S3iG8zFC,0.733,0.436,10,-10.489,1,0.08,0.000573,0.157,0.0976,96.985,202396,4


In [25]:
happy['mood'] = 1

In [26]:
happy.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,mood
0,Post Malone,Twelve Carat Toothache,I Like You (A Happier Song) (with Doja Cat),0O6u0VJ46W86TxN9wgyqDj,0.733,0.67,5,-6.009,1,0.0751,0,0.121,0.472,100.964,192841,4,1
1,NEIKED,Better Days (NEIKED x Mae Muller x Polo G),Better Days (NEIKED x Mae Muller x Polo G),6f5ExP43esnvdKPddwKXJH,0.717,0.671,0,-5.077,0,0.0337,0.000003,0.0921,0.699,110.054,160657,4,1
2,Sabrina Carpenter,emails i can't send,Nonsense,6dgUya35uo964z7GZXM07g,0.74,0.697,8,-4.912,1,0.034,0,0.224,0.732,138.992,163648,4,1
3,Khalid,Skyline,Skyline,1QxIFMs4M8qd5cbJJu80v4,0.733,0.653,11,-6.094,1,0.0391,0.0218,0.184,0.752,119.996,182895,5,1
4,Taylor Swift,Midnights,Lavender Haze,5jQI2r1RdgtuT8S3iG8zFC,0.733,0.436,10,-10.489,1,0.08,0.000573,0.157,0.0976,96.985,202396,4,1


## Playlists Classified as 'Sad'